In [20]:
basePath = "F:\\workspace\\github\\xiedongmingming\\Wav2Lip-GFPGAN"

%cd {basePath}

F:\workspace\github\xiedongmingming\Wav2Lip-GFPGAN


In [21]:
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'

wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

# !wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
# !gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

In [ ]:
# !pip install -r requirements.txt

In [23]:
import os

outputPath = basePath + '/outputs'

inputAudioPath = basePath + '/inputs/kim_audio.mp3'
inputVideoPath = basePath + '/inputs/test.mp4'

print(inputVideoPath)

lipSyncedOutputPath = basePath + '/outputs/result.mp4'

if not os.path.exists(outputPath):
    os.makedirs(outputPath)

!cd $wav2lipFolderName && python inference.py \
    --checkpoint_path checkpoints/wav2lip.pth \
    --face {inputVideoPath} \
    --audio {inputAudioPath} \
    --outfile {lipSyncedOutputPath}

F:\workspace\github\xiedongmingming\Wav2Lip-GFPGAN/inputs/test.mp4
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 655
Extracting raw audio...
(80, 593)
Length of mel chunks: 182
Recovering from OOM error; New batch size: 8
Recovering from OOM error; New batch size: 4
Load checkpoint from: checkpoints/wav2lip.pth
Model loaded


ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable

In [ ]:
# !cd $gfpganFolderName & & python setup.py develop
#
# !wget https: // github.com/TencentARC/GFPGAN/releases/download/v1.3.0 /GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

In [24]:
import cv2

from tqdm import tqdm

from os import path

import os

inputVideoPath = outputPath + '/result.mp4'

unProcessedFramesFolderPath = outputPath + '/frames'

if not os.path.exists(unProcessedFramesFolderPath):

    os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)

numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

fps = vidcap.get(cv2.CAP_PROP_FPS)

print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):

    _, image = vidcap.read()

    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4) + '.jpg'), image)


FPS:  25.0 Frames:  182


100%|██████████| 182/182 [00:08<00:00, 22.44it/s]


In [28]:
!cd $gfpganFolderName && \
    python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None

Traceback (most recent call last):
  File "F:\workspace\github\xiedongmingming\Wav2Lip-GFPGAN\GFPGAN-master\inference_gfpgan.py", line 7, in <module>
    from basicsr.utils import imwrite
ModuleNotFoundError: No module named 'basicsr'


In [ ]:
import os

restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 300

from tqdm import tqdm

for i in tqdm(range(0, len(dir_list), batchSize)):
    img_array = []
    start, end = i, i + batchSize
    print("processing ", start, end)
    for filename in tqdm(dir_list[start:end]):
        filename = restoredFramesPath + filename;
        img = cv2.imread(filename)
        if img is None:
            continue
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

    out = cv2.VideoWriter(processedVideoOutputPath + '/batch_' + str(batch).zfill(4) + '.avi',
                          cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
    batch = batch + 1

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()


In [ ]:
concatTextFilePath = outputPath + "/concat.txt"
concatTextFile = open(concatTextFilePath, "w")
for ips in range(batch):
    concatTextFile.write("file batch_" + str(ips).zfill(4) + ".avi\n")
concatTextFile.close()

concatedVideoOutputPath = outputPath + "/concated_output.avi"
!ffmpeg -y -f concat -i {concatTextFilePath} -c copy {concatedVideoOutputPath}

finalProcessedOuputVideo = processedVideoOutputPath + '/final_with_audio.avi'
!ffmpeg -y -i {concatedVideoOutputPath} -i {inputAudioPath} -map 0 -map 1:a -c:v copy -shortest {finalProcessedOuputVideo}

from google.colab import files

files.download(finalProcessedOuputVideo)